<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Prétraitement des données</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">

# <b>1. Configuration du notebook

In [34]:
# Imports des modules du projet
import config, src

# Imports classiques
import numpy as np
import pandas as pd

# Imports spécifiques
import holidays

In [35]:
# Chargement des données brutes
data = src.load_data(config.RAW_DATA_FILE)

Données chargées avec succès.


In [36]:
data.drop(columns=['rv1', 'rv2'], inplace=True)

In [37]:
rename_dict = {
    'date': 'Date',
    'Appliances': 'Energy',
    'lights': 'Lights',
    'T1': 'Kitchen_T',
    'RH_1': 'Kitchen_H',
    'T2': 'Living_T',
    'RH_2': 'Living_H',
    'T3': 'Laundry_T',
    'RH_3': 'Laundry_H',
    'T4': 'Office_T',
    'RH_4': 'Office_H',
    'T5': 'Bath_T',
    'RH_5': 'Bath_H',
    'T6': 'Outside_T',
    'RH_6': 'Outside_H',
    'T7': 'Ironing_T',
    'RH_7': 'Ironing_H',
    'T8': 'TeenRoom_T',
    'RH_8': 'TeenRoom_H',
    'T9': 'ParentsRoom_T',
    'RH_9': 'ParentsRoom_H',
    'T_out': 'Weather_T',
    'Press_mm_hg': 'Weather_P',
    'RH_out': 'Weather_H',
    'Windspeed': 'Weather_WindSpeed',
    'Visibility': 'Weather_Visibility',
    'Tdewpoint': 'Weather_Dewpoint',
}

data.rename(columns=rename_dict, inplace=True)

In [38]:
src.save_data(data, 'renamed_data')

Données sauvegardées avec succès.


In [39]:
# Conversion des dates au format datetime
data['Date'] = pd.to_datetime(data['Date'])

In [40]:
days = {
    0: 'monday',
    1: 'tuesday',
    2: 'wednesday',
    3: 'thursday',
    4: 'friday',
    5: 'saturday',
    6: 'sunday'
}

days.values()

dict_values(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])

In [41]:
data['day_of_week'] = data['Date'].dt.weekday
data['day_of_week'] = data['day_of_week'].map(days)

one_hot = pd.get_dummies(data['day_of_week'], columns=['day_of_week'], prefix='', prefix_sep='')
one_hot = one_hot[list(days.values())]

data = pd.concat([data.drop(columns=['day_of_week']), one_hot], axis=1)
data[list(days.values())] = data[list(days.values())].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                19735 non-null  datetime64[ns]
 1   Energy              19735 non-null  int64         
 2   Lights              19735 non-null  int64         
 3   Kitchen_T           19735 non-null  float64       
 4   Kitchen_H           19735 non-null  float64       
 5   Living_T            19735 non-null  float64       
 6   Living_H            19735 non-null  float64       
 7   Laundry_T           19735 non-null  float64       
 8   Laundry_H           19735 non-null  float64       
 9   Office_T            19735 non-null  float64       
 10  Office_H            19735 non-null  float64       
 11  Bath_T              19735 non-null  float64       
 12  Bath_H              19735 non-null  float64       
 13  Outside_T           19735 non-null  float64   

In [42]:
data.head()

,Date,Energy,Lights,Kitchen_T,Kitchen_H,Living_T,Living_H,Laundry_T,Laundry_H,Office_T,...,Weather_WindSpeed,Weather_Visibility,Weather_Dewpoint,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,7.000000,63.000000,5.3,1,0,0,0,0,0,0
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,6.666667,59.166667,5.2,1,0,0,0,0,0,0
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,6.333333,55.333333,5.1,1,0,0,0,0,0,0
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,6.000000,51.500000,5.0,1,0,0,0,0,0,0
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,5.666667,47.666667,4.9,1,0,0,0,0,0,0


In [43]:
data['hour'] = data['Date'].dt.hour
data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 24)
data['cos_hour'] = np.cos(2 * np.pi * data['hour'] / 24)

data = data.drop(columns=['hour'])

In [44]:
be_holidays = holidays.Belgium(years=data['Date'].dt.year.unique())
be_holidays = {pd.Timestamp(date): name for date, name in be_holidays.items()}
data['is_holiday'] = data['Date'].isin(be_holidays).astype(int)

n_holidays = data[data['is_holiday'] == 1]['Date'].count()
print(f'Il a {n_holidays} jour(s) férié(s) dans le dataset.')

Il a 6 jour(s) férié(s) dans le dataset.


In [45]:
be_school_holidays_2016 = [
    ('2016-02-08', '2016-02-14'),  # Carnaval
    ('2016-03-28', '2016-04-10'),  # Pâques
    ('2016-07-01', '2016-08-31'),  # Été
    ('2016-10-31', '2016-11-06'),  # Toussaint
    ('2016-12-26', '2017-01-08')   # Noël
]

def is_school_holiday(date, holidays):
    for start, end in holidays:
        if pd.Timestamp(start) <= date <= pd.Timestamp(end):
            return True
    return False

In [46]:
data['is_school_holiday'] = data['Date'].apply(lambda x: is_school_holiday(x, be_school_holidays_2016)).astype(int)

n_school_holidays = len(data.loc[data['is_school_holiday'] == True, 'Date'].dt.date.unique())
print(f'Il y a {n_school_holidays} jour(s) de vacances scolaires dans le dataset.')

Il y a 21 jour(s) de vacances scolaires dans le dataset.


In [47]:
def get_season(date):
    year = date.year
    seasons = {
        0: (pd.Timestamp(year=year, month=3, day=21), pd.Timestamp(year=year, month=6, day=20)), # Printemps
        1: (pd.Timestamp(year=year, month=6, day=21), pd.Timestamp(year=year, month=9, day=20)), # Été
        2: (pd.Timestamp(year=year, month=9, day=21), pd.Timestamp(year=year, month=12, day=20)), # Automne
        3: (pd.Timestamp(year=year, month=12, day=21), pd.Timestamp(year=year+1, month=3, day=20)) # Hiver
        }
    for season, (start, end) in seasons.items():
        if start <= date <= end:
            return season
    return 3 # 01/01 - 20/03

In [48]:
index_to_season = {
    0: 'spring',
    1: 'summer',
    2: 'fall',
    3: 'winter'
}

In [49]:
data['season'] = data['Date'].apply(get_season)

n_seasons = len(data['season'].unique())
print(f'Il y a {n_seasons} différentes dans le dataset.')

Il y a 2 différentes dans le dataset.


In [50]:
data['season'].value_counts()

season
3    9978
0    9757
Name: count, dtype: int64

In [51]:
encode_winter_spring = {
    0: 0,
    3: 1
}

data['season'] = data['season'].replace(encode_winter_spring)

In [52]:
data['season'].value_counts()

season
1    9978
0    9757
Name: count, dtype: int64

In [53]:
data.head()

,Date,Energy,Lights,Kitchen_T,Kitchen_H,Living_T,Living_H,Laundry_T,Laundry_H,Office_T,...,wednesday,thursday,friday,saturday,sunday,sin_hour,cos_hour,is_holiday,is_school_holiday,season
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1


In [54]:
data.duplicated().value_counts()

False    19735
Name: count, dtype: int64

In [55]:
src.save_data(data, 'processed_data')

Données sauvegardées avec succès.
